In [1]:
import numpy as np
import pandas as pd

In [2]:
train_df  = pd.read_csv('earthquake/earthquake_train.csv')
test_df  = pd.read_csv('earthquake/earthquake_test.csv')

In [3]:
train_df.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,3
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,2
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,3
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,2
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,3


In [4]:
train_df.groupby('geo_level_3_id').agg({'geo_level_3_id': 'count'})

,geo_level_3_id
geo_level_3_id,
0,2
1,6
3,9
5,14
6,21
...,...
12562,3
12563,24
12564,6


In [5]:
#df = df.sample(frac=0.02)

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 40 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   building_id                             260601 non-null  int64 
 1   geo_level_1_id                          260601 non-null  int64 
 2   geo_level_2_id                          260601 non-null  int64 
 3   geo_level_3_id                          260601 non-null  int64 
 4   count_floors_pre_eq                     260601 non-null  int64 
 5   age                                     260601 non-null  int64 
 6   area_percentage                         260601 non-null  int64 
 7   height_percentage                       260601 non-null  int64 
 8   land_surface_condition                  260601 non-null  object
 9   foundation_type                         260601 non-null  object
 10  roof_type                               260601 non-null 

In [7]:
mean_damage_grade = train_df['damage_grade'].mean()

In [8]:
res2 = []
for level_1_id in train_df['geo_level_1_id'].unique():
    
    df_tmp = train_df[train_df['geo_level_1_id']==level_1_id]
    
    count_level_up = df_tmp['damage_grade'].count()
    mean_level_up = df_tmp['damage_grade'].mean()
    
    std_level_up = 0.8
    if count_level_up > 1:
        std_level_up  = df_tmp['damage_grade'].std()+ 0.001
    
    for level_2_id in df_tmp['geo_level_2_id'].unique():
        df_tmp2 = df_tmp[df_tmp['geo_level_2_id']==level_2_id]
        count_sub = df_tmp2.shape[0]
        mean_sub = df_tmp2['damage_grade'].mean()
        
        std_sub = 0.8
        denom = 1
        lambd = 0.2
        if count_level_up < 5 and count_level_up < 5:
            lambd=0.5
        elif count_sub > 5:
            std_sub  = df_tmp2['damage_grade'].std() + 0.001
            denom = ((std_sub/std_level_up) + count_sub)
            lambd = 1
            if denom != 0:
                lambd = count_sub / ((std_sub/std_level_up) + count_sub)

        s = (lambd * mean_sub) + ((1-lambd) * mean_level_up)

        res2.append({
            'geo_level_1_id': level_1_id,
            'geo_level_2_id': level_2_id,
            'count_level_up': count_level_up,
            'mean_level_up': mean_level_up,
            'std_level_up': std_level_up,
            'count_sub': count_sub,
            'mean_sub': mean_sub,
            'std_sub': std_sub,
            'lambd': lambd,
            'denom': denom,
            'lvl2_s': s,
        })

In [9]:
lvl2s = pd.DataFrame(res2)
lvl2s

,geo_level_1_id,geo_level_2_id,count_level_up,mean_level_up,std_level_up,count_sub,mean_sub,std_sub,lambd,denom,lvl2_s
0,6,487,24381,2.161724,0.556431,270,2.740741,0.448429,0.997024,270.805902,2.739018
1,6,706,24381,2.161724,0.556431,216,2.115741,0.321657,0.997331,216.578071,2.115863
2,6,1210,24381,2.161724,0.556431,299,2.488294,0.501701,0.996994,299.901641,2.487312
3,6,724,24381,2.161724,0.556431,529,2.274102,0.488058,0.998345,529.877123,2.273916
4,6,139,24381,2.161724,0.556431,114,2.280702,0.452326,0.992920,114.812905,2.279859
...,...,...,...,...,...,...,...,...,...,...,...
1409,29,637,396,2.078283,0.336922,1,2.000000,0.800000,0.200000,1.000000,2.062626
1410,29,554,396,2.078283,0.336922,8,2.250000,0.463910,0.853160,9.376906,2.224785
1411,29,989,396,2.078283,0.336922,1,2.000000,0.800000,0.200000,1.000000,2.062626
1412,29,250,396,2.078283,0.336922,3,2.000000,0.800000,0.200000,1.000000,2.062626


In [10]:
res3 = []
for level_2_id in train_df['geo_level_2_id'].unique():
    
    df_tmp = train_df[train_df['geo_level_2_id']==level_2_id]
    
    count_level_up = df_tmp['damage_grade'].count()
    mean_level_up = df_tmp['damage_grade'].mean()
    
    std_level_up = 0.8
    if count_level_up > 1:
        std_level_up  = df_tmp['damage_grade'].std()+ 0.001
    
    for level_3_id in df_tmp['geo_level_3_id'].unique():
        df_tmp2 = df_tmp[df_tmp['geo_level_3_id']==level_3_id]
        count_sub = df_tmp2.shape[0]
        mean_sub = df_tmp2['damage_grade'].mean()
        
        lambd = 0.2
        if count_level_up < 5 and count_level_up < 5:
            lambd=0.5
        elif count_sub > 5:
            std_sub  = df_tmp2['damage_grade'].std() + 0.001
            denom = ((std_sub/std_level_up) + count_sub)
            lambd = 1
            if denom != 0:
                lambd = count_sub / ((std_sub/std_level_up) + count_sub)

        s = (lambd * mean_sub) + ((1-lambd) * mean_level_up)

        res3.append({
            'geo_level_2_id': level_2_id,
            'geo_level_3_id': level_3_id,
            'count_level_up': count_level_up,
            'mean_level_up': mean_level_up,
            'std_level_up': std_level_up,
            'count_sub': count_sub,
            'mean_sub': mean_sub,
            'std_sub': std_sub,
            'lambd': lambd,
            'denom': denom,
            'lvl3_s': s,
        })

In [11]:
lvl3s = pd.DataFrame(res3)
lvl3s

,geo_level_2_id,geo_level_3_id,count_level_up,mean_level_up,std_level_up,count_sub,mean_sub,std_sub,lambd,denom,lvl3_s
0,487,12198,270,2.740741,0.448429,37,2.837838,0.374684,0.977916,37.835548,2.835694
1,487,7439,270,2.740741,0.448429,52,2.750000,0.438237,0.981553,52.977272,2.749829
2,487,835,270,2.740741,0.448429,20,2.550000,0.511418,0.946053,21.140465,2.560290
3,487,2908,270,2.740741,0.448429,21,2.428571,0.508093,0.948807,22.133050,2.444552
4,487,382,270,2.740741,0.448429,18,2.833333,0.384482,0.954533,18.857399,2.829123
...,...,...,...,...,...,...,...,...,...,...,...
11590,975,4341,1,2.000000,0.800000,1,2.000000,0.001000,0.500000,8.003314,2.000000
11591,771,6878,1,3.000000,0.800000,1,3.000000,0.001000,0.500000,8.003314,3.000000
11592,77,8003,1,3.000000,0.800000,1,3.000000,0.001000,0.500000,8.003314,3.000000
11593,115,10136,1,1.000000,0.800000,1,1.000000,0.001000,0.500000,8.003314,1.000000


In [12]:
new_df = pd.merge(train_df, lvl2s[['geo_level_1_id', 'geo_level_2_id', 'lvl2_s']], how='left', on=['geo_level_1_id', 'geo_level_2_id'])
new_df['lvl2_s'] = new_df['lvl2_s'].fillna(mean_damage_grade)
new_df
new_df = pd.merge(new_df, lvl3s[['geo_level_2_id', 'geo_level_3_id', 'lvl3_s']], how='left', on=['geo_level_2_id', 'geo_level_3_id'])
new_df['lvl3_s'] = new_df['lvl3_s'].fillna(mean_damage_grade)
new_df['s_avg'] = (new_df['lvl2_s'] + new_df['lvl3_s']) / 2
new_df

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,lvl2_s,lvl3_s,s_avg
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,3,2.739018,2.835694,2.787356
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,2,2.487427,2.083933,2.285680
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,3,2.518780,2.580493,2.549637
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,2,2.106944,2.097128,2.102036
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,3,2.348737,2.368699,2.358718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,688636,25,1335,1621,1,55,6,3,n,r,...,0,0,0,0,0,0,2,1.932212,2.066475,1.999343
260597,669485,17,715,2060,2,0,6,5,t,r,...,0,0,0,0,0,0,3,2.930712,2.979069,2.954890
260598,602512,17,51,8163,3,55,6,7,t,r,...,0,0,0,0,0,0,3,2.894634,2.822544,2.858589
260599,151409,26,39,1851,2,10,14,6,t,r,...,0,0,0,0,0,0,2,1.532241,1.791535,1.661888


In [13]:
new_df.to_csv('earthquake/earthquake_train_lvls.csv', index=False)

In [14]:
new_df = pd.merge(test_df, lvl2s[['geo_level_1_id', 'geo_level_2_id', 'lvl2_s']], how='left', on=['geo_level_1_id', 'geo_level_2_id'])
new_df['lvl2_s'] = new_df['lvl2_s'].fillna(mean_damage_grade)
new_df
new_df = pd.merge(new_df, lvl3s[['geo_level_2_id', 'geo_level_3_id', 'lvl3_s']], how='left', on=['geo_level_2_id', 'geo_level_3_id'])
new_df['lvl3_s'] = new_df['lvl3_s'].fillna(mean_damage_grade)
new_df['s_avg'] = (new_df['lvl2_s'] + new_df['lvl3_s']) / 2
new_df.to_csv('earthquake/earthquake_test_lvls.csv', index=False)
new_df

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,lvl2_s,lvl3_s,s_avg
0,300051,17,596,11307,3,20,7,6,t,r,...,0,0,0,0,0,0,0,2.705730,2.636152,2.670941
1,99355,6,141,11987,2,25,13,5,t,r,...,0,0,0,0,0,0,0,2.180781,2.000052,2.090416
2,890251,22,19,10044,2,5,4,5,t,r,...,0,0,0,0,0,0,0,2.171778,2.340000,2.255889
3,745817,26,39,633,1,0,19,3,t,r,...,0,0,0,0,0,0,0,1.532241,1.437918,1.485079
4,421793,17,289,7970,3,15,8,7,t,r,...,0,0,0,0,0,0,0,2.689712,2.817291,2.753502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86863,310028,4,605,3623,3,70,20,6,t,r,...,0,0,0,0,0,0,0,2.122769,1.868420,1.995595
86864,663567,10,1407,11907,3,25,6,7,n,r,...,0,0,0,0,0,0,0,2.337190,2.706143,2.521666
86865,1049160,22,1136,7712,1,50,3,3,t,r,...,0,0,0,0,0,0,0,2.019626,1.915678,1.967652
86866,442785,6,1041,912,2,5,9,5,t,r,...,0,0,0,0,0,0,0,2.295512,2.138618,2.217065
